In [ ]:
# # stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# isort: off
# stdlib
from os import environ as env

# syft absolute
import syft as sy
from syft import get_helpers  # noqa: F401

# third party
from email_helpers import get_email_server
# isort: on

# Launch & login

In [ ]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server(reset=True)

In [ ]:
root_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

# Create new admin client

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [ ]:
# create
root_client.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

# We cannot delete the root client

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

In [ ]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [ ]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

In [ ]:
root_client.users.delete(new_user_id2)

In [ ]:
# import os
# from syft.client.client import SyftClient
# from syft.service.user.user_roles import ServiceRole
# import ipykernel
# import json
# from pathlib import Path
# import uuid
# from syft.util.util import get_root_data_path
# from syft import SyftError, SyftException

# NB_STATE_DIRECTORY = "nb_states"

# def current_nbname() -> Path:
#     """Get the """
#     curr_kernel_file = Path(ipykernel.get_connection_file())
#     kernel_file = json.loads(curr_kernel_file.read_text())
#     nb_name = kernel_file['jupyter_session']
#     return Path(nb_name)


# def get_or_create_state_dir(filename: str) -> Path:
#     nb_path = get_root_data_path() / NB_STATE_DIRECTORY
#     filepath = nb_path / filename
#     if not filepath.exists():
#         filepath.mkdir(parents=True, exist_ok=True)
#     return filepath


# def state_dir_for_nb() -> Path:
#     nb_name = current_nbname()
#     filename = nb_name.name.split(".json")[0]
#     return get_or_create_state_dir(filename=nb_name.name)


# def is_admin(client: SyftClient) -> bool:
#     return client._SyftClient__user_role == ServiceRole.ADMIN

# def checkpoint_db(client: SyftClient, root_email: str | None = None, root_pwd: str | None = None) -> None:
#     migration_data = client.get_migration_data(include_blobs=True)
#     if isinstance(migration_data, SyftError):
#         raise SyftException(message=migration_data.message)

#     state_dir = state_dir_for_nb()
#     state_dir_server = state_dir / client.id.to_string()
#     migration_data.save(path=state_dir_server/"migration.blob", yaml_path=state_dir_server/"migration.yaml")
#     return state_dir_server

In [ ]:
# third party
from checkpoint import checkpoint_db

In [ ]:
checkpoint_db(root_client)

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()